In [3]:
import gurobipy as gp
from gurobipy import GRB

def solve_explanation_m_1(candidate_sup, candidate_inf, notes_sup, notes_inf, poids, matieres):
    """
    Résout le problème d'explication de type (m-1).
    Plusieurs 'Pros' peuvent compenser un seul 'Con'.
    """
    # 1. Calcul des deltas
    deltas = []
    for k in range(len(matieres)):
        val = poids[k] * (notes_sup[k] - notes_inf[k])
        deltas.append(val)
        
    P = [i for i, d in enumerate(deltas) if d > 0]
    C = [j for j, d in enumerate(deltas) if d < 0]
    
    print(f"\nComparaison {candidate_sup} > {candidate_inf}")
    print(f"Pros: {[(matieres[i], deltas[i]) for i in P]}")
    print(f"Cons: {[(matieres[j], deltas[j]) for j in C]}")

    if not C:
        print("Aucun désavantage à expliquer.")
        return

    # 2. Modèle Gurobi
    m = gp.Model(f"Explication_m_1_{candidate_sup}_{candidate_inf}")
    m.setParam('OutputFlag', 0)

    # Variables x[i, j] : Pro i aide à couvrir Con j
    x = {}
    for i in P:
        for j in C:
            x[i, j] = m.addVar(vtype=GRB.BINARY, name=f"x_{matieres[i]}_{matieres[j]}")

    # C1: Chaque Pro utilisé au max 1 fois
    for i in P:
        m.addConstr(gp.quicksum(x[i, j] for j in C) <= 1, name=f"disjoint_{matieres[i]}")

    # C2: Chaque Con doit être compensé (Somme Pros + Con >= 0)
    # On utilise une tolérance très faible ou nulle. Ici >= 0 accepte l'égalité parfaite.
    for j in C:
        m.addConstr(gp.quicksum(deltas[i] * x[i, j] for i in P) + deltas[j] >= 0, 
                    name=f"cover_{matieres[j]}")

    # 3. Résolution
    m.optimize()

    # 4. Analyse
    if m.Status == GRB.OPTIMAL:
        print(">>> SUCCÈS : Explication (m-1) trouvée.")
        for j in C:
            # Récupérer les Pros assignés à ce Con j
            assigned_pros = [matieres[i] for i in P if x[i, j].X > 0.5]
            val_pros = sum(deltas[i] for i in P if x[i, j].X > 0.5)
            print(f"  - Le désavantage '{matieres[j]}' ({deltas[j]}) est compensé par {assigned_pros} (Total: +{val_pros})")
    elif m.Status == GRB.INFEASIBLE:
        print(">>> ÉCHEC : Pas d'explication (m-1) possible.")
    else:
        print(f"Statut solveur : {m.Status}")

# --- Données ---
matieres = ["Anatomie", "Biologie", "Chirurgie", "Diagnostic", "Epidemiologie", "Forensic", "Génétique"]
poids = [8, 7, 7, 6, 6, 5, 6]

# Notes (Tableau Page 2)
# y: 81, 81, 75, 63, 67, 88, 95
# z: 74, 89, 74, 81, 68, 84, 79
# t: 74, 71, 84, 91, 77, 76, 73

notes_y = [81, 81, 75, 63, 67, 88, 95]
notes_z = [74, 89, 74, 81, 68, 84, 79]
notes_t = [74, 71, 84, 91, 77, 76, 73]

# Test 1 : y > z
solve_explanation_m_1("y", "z", notes_y, notes_z, poids, matieres)

# Test 2 : z > t
solve_explanation_m_1("z", "t", notes_z, notes_t, poids, matieres)


Comparaison y > z
Pros: [('Anatomie', 56), ('Chirurgie', 7), ('Forensic', 20), ('Génétique', 96)]
Cons: [('Biologie', -56), ('Diagnostic', -108), ('Epidemiologie', -6)]
>>> SUCCÈS : Explication (m-1) trouvée.
  - Le désavantage 'Biologie' (-56) est compensé par ['Anatomie'] (Total: +56)
  - Le désavantage 'Diagnostic' (-108) est compensé par ['Forensic', 'Génétique'] (Total: +116)
  - Le désavantage 'Epidemiologie' (-6) est compensé par ['Chirurgie'] (Total: +7)

Comparaison z > t
Pros: [('Biologie', 126), ('Forensic', 40), ('Génétique', 36)]
Cons: [('Chirurgie', -70), ('Diagnostic', -60), ('Epidemiologie', -54)]
>>> ÉCHEC : Pas d'explication (m-1) possible.
